# Advent of Code - 2024 - Day 17 - Problem 2

https://adventofcode.com/2024/day/17

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day17.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Emulator Class

In [2]:
class Emulator:

    # Instructions
    ADV = 0
    BDV = 6
    BST = 2
    BXC = 4
    BXL = 1
    CDV = 7
    JNZ = 3
    OUT = 5

    # Registers
    A = 0
    B = 1
    C = 2

    def __init__(self):
        self._registers = [0, 0, 0]
        self._ip = 0
        self._code = []
        self._output = []
        self._trace = False

    def load_program(self, program):
        for register in range(3):
            line = program[register]
            idx = line.index(":")
            value = int(line[idx + 1 :])
            self._registers[register] = value

        line = program[4]
        idx = line.index(":")
        self._code = list(map(int, line[idx + 1 :].split(",")))

    def execute(self):
        opcode = self._get_opcode()
        if opcode == None:
            return False

        if opcode == self.ADV:
            self._execute_adv()
        elif opcode == self.BDV:
            self._execute_bdv()
        elif opcode == self.BST:
            self._execute_bst()
        elif opcode == self.BXC:
            self._execute_bxc()
        elif opcode == self.BXL:
            self._execute_bxl()
        elif opcode == self.CDV:
            self._execute_cdv()
        elif opcode == self.JNZ:
            self._execute_jnz()
        elif opcode == self.OUT:
            self._execute_out()

        return True

    def get_output(self):
        return self._output

    def set_trace(self, trace):
        self._trace = trace

    #
    # State Management
    #

    def _advance_ip(self):
        self._ip += 2

    def _set_ip(self, ip):
        self._ip = ip

    def _get_opcode(self):
        if self._ip >= len(self._code):
            return None
        return self._code[self._ip]

    def _get_register(self, register):
        return self._registers[register]

    def _set_register(self, register, value):
        self._registers[register] = value

    def _write_output(self, value):
        self._output.append(value)

    def _get_literal_operand(self):
        return self._code[self._ip + 1]

    def _get_combo_operand(self):
        value = self._code[self._ip + 1]
        if value <= 3:
            return value
        elif value == 4:
            return self._get_register(self.A)
        elif value == 5:
            return self._get_register(self.B)
        elif value == 6:
            return self._get_register(self.C)
        else:
            raise Exception(f"Illegal value {value}")

    def _get_combo_description(self):
        value = self._code[self._ip + 1]
        if value <= 3:
            return str(value)
        elif value == 4:
            return "A"
        elif value == 5:
            return "B"
        elif value == 6:
            return "C"
        else:
            raise Exception(f"Illegal value {value}")

    #
    # Instructions
    #

    def _execute_adv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:ADV - A({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.A, result)
        self._advance_ip()

    def _execute_bdv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:BDV - B({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bst(self):
        operand = self._get_combo_operand()
        result = operand % 8

        if self._trace:
            print(f"{self._ip}:BST - B({oct(result)}) = {self._get_combo_description()}({oct(operand)}) % 8")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bxc(self):
        register_b = self._get_register(self.B)
        register_c = self._get_register(self.C)
        result = register_b ^ register_c

        if self._trace:
            print(f"{self._ip}:BXC - B({oct(result)}) = B({oct(register_b)}) ^ C({oct(register_c)})")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bxl(self):
        register_b = self._get_register(self.B)
        value = self._get_literal_operand()
        result = register_b ^ value

        if self._trace:
            print(f"{self._ip}:BXL - B({oct(result)}) = B({oct(register_b)}) ^ {oct(value)}")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_cdv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:CDV - C({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.C, result)
        self._advance_ip()

    def _execute_jnz(self):
        register = self._get_register(self.A)
        value = self._get_literal_operand()

        if self._trace:
            print(f"{self._ip}:JNZ - A({oct(register)}) -> {value}")

        if register == 0:
            self._advance_ip()
        else:
            self._set_ip(value)

    def _execute_out(self):
        value = self._get_combo_operand()
        result = value % 8

        if self._trace:
            print(f"{self._ip}:OUT - {oct(result)} = {self._get_combo_description()}({oct(value)}) % 8")

        self._write_output(result)
        self._advance_ip()

## Define find_input / find_input_2

Iteratively searches for possible inputs starting with right-most value.

In [3]:
def find_input(starting_with, producing):
    for digit in range(8):
        input = starting_with + str(digit)
        emulator = Emulator()
        emulator.load_program(DATA)
        emulator._set_register(emulator.A,int(input,8) )
        while emulator.execute():
            pass
        result = "".join(map(str, emulator.get_output()))
        if result == producing:
            yield input


def find_input_2(producing):
    if producing == "": yield ""
    else:
        for starting_with in find_input_2(producing[1:]):
            for input in find_input(starting_with, producing):
                yield input



## Search for Valid Input

In [4]:
for result in find_input_2("2411750314455530"):
    print(int(result,8))
    break

202322348616234
